In [2]:
import requests
import re
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

citys = ['losangeles','chicago','houston','phoenix', 'philadelphia', 'sanantonio', 'sandiago', 'dallas','sanjose', 'austin', 'jacksonville', 'fortworth', 'columbus', 'charlotte', 'indianapolis', '	fanfrancisco', 'settle', 'denver', 'washington', 'nashville'] #*****change this city to what you want!!!!*****

#just grabbing the first 20 pages
#feel free to make this prettier
#add headers in case you use chromedriver (captchas are no fun); namely used for chromedriver
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
} 

data_list = []
for city in citys:
    for i in range(1, 21):
        url = 'https://www.zillow.com/homes/for_sale/'+city+'/' + str(i) + '_p/'
        with requests.Session() as s:
            r = s.get(url, headers=req_headers)
        #     try:
            data = json.loads(re.search(r'!--(\{"queryState".*?)-->', r.text).group(1))
            if data is None:
                pass
            # except AttributeError:
            #     data = json.loads(re.search(r'!--(\{"queryState".*?)-->', r1.text))      
        data_list.append(data)   



df = pd.DataFrame()

def make_frame(frame):
    for i in data_list:
        for item in i['cat1']['searchResults']['listResults']:
            frame = frame.append(item, ignore_index=True)
    return frame

df = make_frame(df)
    
#drop cols
# df = df.drop('hdpData', 1) #remove this line to see a whole bunch of other random cols, in dict format

#drop dupes
df = df.drop_duplicates(subset='zpid', keep="last")

#filters
df['zestimate'] = df['zestimate'].fillna(-1)
df['best_deal'] = df['unformattedPrice'] - df['zestimate']
df = df.sort_values(by='best_deal',ascending=True)

print('shape:', df.shape)
display(df[['id','address','beds','baths','area','price','zestimate','best_deal','hdpData']].head(20))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [3]:
data_list

[{'queryState': {'usersSearchTerm': 'Los Angeles, CA',
   'mapBounds': {'west': -118.668176,
    'east': -118.155289,
    'south': 33.703652,
    'north': 34.337306},
   'regionSelection': [{'regionId': 12447, 'regionType': 6}],
   'isMapVisible': True,
   'filterState': {'sortSelection': {'value': 'globalrelevanceex'},
    'isAllHomes': {'value': True}}},
  'filterDefinitions': {'keywords': {'id': 'keywords',
    'shortId': 'att',
    'labels': {'default': 'Keywords', 'tracking': 'Keyword'},
    'sortOrder': 2,
    'type': 'String',
    'defaultValue': {'value': ''},
    'exposedPillEnabled': True},
   'isPublicSchool': {'id': 'isPublicSchool',
    'shortId': 'schp',
    'labels': {'default': 'Public'},
    'type': 'Boolean',
    'defaultValue': {'value': True}},
   'isCityView': {'id': 'isCityView',
    'shortId': 'cityv',
    'labels': {'default': 'City'},
    'type': 'Boolean',
    'defaultValue': {'value': False}},
   'isWaterfront': {'id': 'isWaterfront',
    'shortId': 'wat',
  

In [2]:
df.head()

,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,price,...,list,relaxed,info1String,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription,lotAreaString,info2String,best_deal
349,31497883,31497883,1624407,https://photos.zillowstatic.com/fp/1dd8629c727...,True,https://www.zillow.com/homedetails/196-W-Houst...,FOR_SALE,House for sale,$,"$13,800,000",...,True,False,NaN,True,2022-11-13T13:00:00,2022-11-13T15:00:00,Open House - 1:00 - 3:00 PM,NaN,NaN,-1125200
109,30601845,30601845,1628422,https://photos.zillowstatic.com/fp/c52c6486dea...,True,https://www.zillow.com/homedetails/361-Vernon-...,FOR_SALE,Multi-family home for sale,$,"$500,000",...,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-919400
176,89027543,89027543,11012604,https://photos.zillowstatic.com/fp/ff76eaadadf...,True,https://www.zillow.com/homedetails/10727-155th...,FOR_SALE,House for sale,$,"$300,000",...,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-698300
40,113087202,113087202,1626948,https://photos.zillowstatic.com/fp/73b25ddac5d...,True,https://www.zillow.com/homedetails/100-W-57th-...,FOR_SALE,Condo for sale,$,"$349,000",...,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-680100
90,30785574,30785574,11114387,https://photos.zillowstatic.com/fp/05c2818fb87...,True,https://www.zillow.com/homedetails/1792-E-54th...,FOR_SALE,Multi-family home for sale,$,"$250,000",...,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-564900


In [3]:
pd.set_option('display.max_columns', None)

#flatten
def flatten_json(nested_json, exclude=['']):
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

#merge frames
A = pd.DataFrame([flatten_json(x) for x in df['hdpData']])
df['id'] = df['id'].astype(int)
A['homeInfo_zpid'] = A['homeInfo_zpid'].astype(int)
df = pd.concat([df,A], axis=1).fillna(-1)
print(df.shape)
df.head()

(520, 97)


,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,price,unformattedPrice,address,addressStreet,addressCity,addressState,addressZipcode,isUndisclosedAddress,beds,baths,area,latLong,isZillowOwned,variableData,badgeInfo,hdpData,isSaved,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,zestimate,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,info3String,brokerName,hasAdditionalAttributions,isFeaturedListing,availabilityDate,list,relaxed,info1String,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription,lotAreaString,info2String,best_deal,homeInfo_zpid,homeInfo_streetAddress,homeInfo_zipcode,homeInfo_city,homeInfo_state,homeInfo_latitude,homeInfo_longitude,homeInfo_price,homeInfo_datePriceChanged,homeInfo_bathrooms,homeInfo_bedrooms,homeInfo_livingArea,homeInfo_homeType,homeInfo_homeStatus,homeInfo_daysOnZillow,homeInfo_isFeatured,homeInfo_shouldHighlight,homeInfo_zestimate,homeInfo_rentZestimate,homeInfo_listing_sub_type_is_openHouse,homeInfo_listing_sub_type_is_FSBA,homeInfo_openHouse,homeInfo_priceReduction,homeInfo_isUnmappable,homeInfo_isPreforeclosureAuction,homeInfo_homeStatusForHDP,homeInfo_priceForHDP,homeInfo_open_house_info_open_house_showing_0_open_house_start,homeInfo_open_house_info_open_house_showing_0_open_house_end,homeInfo_priceChange,homeInfo_isNonOwnerOccupied,homeInfo_isPremierBuilder,homeInfo_isZillowOwned,homeInfo_currency,homeInfo_country,homeInfo_taxAssessedValue,homeInfo_lotAreaValue,homeInfo_lotAreaUnit,homeInfo_unit,homeInfo_listing_sub_type_is_bankOwned,homeInfo_isRentalWithBasePrice,homeInfo_listing_sub_type_is_forAuction,homeInfo_videoCount,homeInfo_open_house_info_open_house_showing_1_open_house_start,homeInfo_open_house_info_open_house_showing_1_open_house_end,homeInfo_listing_sub_type_is_newHome,homeInfo_newConstructionType
349,31497883,31497883,1624407,https://photos.zillowstatic.com/fp/1dd8629c727...,True,https://www.zillow.com/homedetails/196-W-Houst...,FOR_SALE,House for sale,$,"$13,800,000",13800000,"196 W Houston St, New York, NY 10014",196 W Houston St,New York,NY,10014,False,3,8.0,7200,"{'latitude': 40.728725, 'longitude': -74.00385}",False,"{'type': 'OPEN_HOUSE', 'text': 'Open: Sun. 1-3...",-1,"{'homeInfo': {'zpid': 31497883, 'streetAddress...",False,False,False,ForSale,For Sale (Broker),14925200,False,False,False,False,-1,Listing by: Douglas Elliman,True,False,-1,True,False,-1,True,2022-11-13T13:00:00,2022-11-13T15:00:00,Open House - 1:00 - 3:00 PM,-1,-1,-1125200,2061667925,2257 Cayuga Dr,14304,Niagara Falls,NY,43.097588,-78.926704,250000.0,1.667981e+12,2.0,3.0,1417.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,-1.0,-1.0,-1,True,-1,"$25,000 (Nov 9)",False,False,FOR_SALE,250000.0,-1.000000e+00,-1.000000e+00,-25000.0,True,False,False,USD,USA,-1.0,0.5294,acres,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1,-1
109,30601845,30601845,1628422,https://photos.zillowstatic.com/fp/c52c6486dea...,True,https://www.zillow.com/homedetails/361-Vernon-...,FOR_SALE,Multi-family home for sale,$,"$500,000",500000,"361 Vernon Ave, Brooklyn, NY 11206",361 Vernon Ave,Brooklyn,NY,11206,False,5,3.0,2880,"{'latitude': 40.696304, 'longitude': -73.93657}",False,"{'type': 'DAYS_ON', 'text': '20 days on Zillow'}",-1,"{'homeInfo': {'zpid': 30601845, 'streetAddress...",False,False,False,ForSale,For Sale (Broker),1419400,False,False,False,False,-1,Listing by: DeRuscio Commercial LLC,True,False,-1,True,False,-1,-1,-1,-1,-1,-1,-1,-919400,31666835,2661 E Genesee St,13224,Syracuse,NY,43.044357,-76.109795,299000.0,-1.000000e+00,4.0,4.0,3951.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,301800.0,2499.0,-1,True,-1,-1,False,False,FOR_SALE,299000.0,-1.000000e+00,-1.000000e+00,-1.0,True,False,False,USD,USA,194631.0,0.4021,acres,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1,-1
176,89027543,89027543,11012604,https://photos.zillowstatic.com/fp/ff76eaadadf...,True,https://www.zillow.com/homedetails/10727-155th...,FOR_SALE,House for sale,$,"$300,000",300000,"10727 155th St, Jamaica, NY 11433",10727 155th St,Jama